In [1]:
import pandas as pd, string, re

In [2]:
df1=pd.read_csv('../triple_long_sf_Vax.csv', header=0, encoding='ISO-8859-1', engine='python')

In [3]:
len(df1)

13806

In [4]:
df1=df1.drop_duplicates(subset=['triple_string'], keep='first')

In [5]:
len(df1)

13806

In [6]:
df1=df1[(df1['sentiment']==0) | (df1['sentiment']==1)]

In [7]:
len(df1)

5423

In [39]:
####################################################################################

In [40]:
import nltk
import re
import numpy as np
import contractions
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()
stop_words = nltk.corpus.stopwords.words('english')
########################################################################################
def normalize_document(doc):
    #nns=["NN", "NNS", "NNP", "NNPS"]
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)

    tokens = nltk.word_tokenize(doc)

    filtered_tokenss = [token for token in tokens if (token not in stop_words) and (len(token)>2)]
    filtered_tokenss = [lemmatizer.lemmatize(token, pos ="n") for token in filtered_tokenss ]

    ######################################################################
    doc = ' '.join(filtered_tokenss)

    return doc

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cl524\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [41]:
normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(list(df1['subject']))
print(len(norm_corpus))
tokenized_docs = [(doc.split())[-2:] for doc in norm_corpus]

processedSub=[]

print(len(tokenized_docs))
for t in tokenized_docs:
    processedSub.append(" ".join(t))
    
print(processedSub[:10])
print(len(processedSub))


926
926
['vaccine', 'efficacy', 'vaccine', 'mike penny', 'covid vaccine', '', 'presidentelect joebiden', 'good news', 'good news', 'mrna vaccine']
926


In [42]:
df1['processedSub']=processedSub

df1=df1[df1['processedSub']!='']

print(len(df1))

753


In [43]:
from pandas import DataFrame
dout=DataFrame()


dout=df1

dout=dout#.sort_values(by = ['dateTime'], ascending = [True])
print(len(dout))

753


In [44]:
df1=dout

In [45]:
len(df1)

753

In [46]:
salience=[]

for f, r in zip(list(df1['followers']), list(df1['retweets'])):
    if f>0 and r>0:
        salience.append(int(f)+int(r)*707)
    elif r==0:
        salience.append(int(f))
    else:
        salience.append(0)

df1['salienceScore']=salience

In [47]:
print(df1['salienceScore'][:5], df1['followers'][:5], df1['retweets'][:5])

15      240308
71       15399
84     7993639
85       15378
110     811811
Name: salienceScore, dtype: int64 15      240308
71       15399
84     7991518
85       15378
110     809690
Name: followers, dtype: int64 15     0
71     0
84     3
85     0
110    3
Name: retweets, dtype: int64


In [64]:
dout=DataFrame()
dout=df1.sort_values(["salienceScore"], ascending=[False]).head(18)
dout=dout.sort_values(by=['postInd', 'sentenceInd', 'tripleInd'], ascending=[True, True, True])

In [65]:
dout2=dout#.drop_duplicates(subset=['postInd', 'sentenceInd'], keep='first')
print(len(dout2))

from pandas import DataFrame
dout_sent=DataFrame()#(columns=['dateTime', 'postInd', 'sentenceInd', 'tripleInd', 'subject', 'relation', 'object', 'sentence', 'userId', 'followers', 'retweets', 'salienceScore', 'vacccines'])

dout_sent['dateTime'] = list(dout2['dateTime'])
dout_sent['postInd'] = list(dout2['postInd'])
dout_sent['sentenceInd'] = list(dout2['sentenceInd'])
dout_sent['tripleInd'] = list(dout2['tripleInd'])
dout_sent['subject'] = list(dout2['processedSub'])
dout_sent['relation'] = list(dout2['relation'])
dout_sent['object'] = list(dout2['object'])
dout_sent['sentiment'] = list(dout2['sentiment'])
dout_sent['country'] = list(dout2['country'])
dout_sent['sentence'] = list(dout2['sentence'])
dout_sent['post'] = list(dout2['post'])
dout_sent['userId'] = list(dout2['userId'])
dout_sent['followers'] = list(dout2['followers'])
dout_sent['retweets'] = list(dout2['retweets'])
dout_sent['salienceScore'] = list(dout2['salienceScore'])
dout_sent['vaccines'] = list(dout2['vaccines'])

dout_sent['tripleS']=dout_sent['subject']+" "+dout_sent['relation']+" "+dout_sent['object']

dout_sent.to_csv('tripleSent_summary_world_POS.csv', index=None)

18


In [66]:
print(len(list(set(list(dout_sent['userId'])))))
print(len(list(set(list(dout_sent['sentence'])))))

6
16


In [67]:
(dout_sent.groupby(['vaccines']).sum(['retweets'])).drop(['userId', 'postInd', 'sentenceInd', 'tripleInd', 'salienceScore'], axis=1)

,sentiment,followers,retweets
vaccines,,,
covaxin,30,118705315,619
moderna pfizer,3,13115701,10
sinopharm,6,26231740,212
sinovac,12,52463504,17
sputnik,3,1015996,12925


In [68]:
###########################################################################################################

In [69]:
dk=pd.read_csv("tripleSent_summary_world_POS.csv", header=0, encoding='ISO-8859-1', engine='python')

In [70]:
len(dk)

18

In [71]:
from pandas import DataFrame
dout=DataFrame()


dk=dk.drop_duplicates(subset=['postInd', 'sentenceInd'], keep='first')
print(len(dk))

sentences=list(dk['sentence'])
summary_sent=[]
for p in sentences:
    p=str(p)
    p=re.sub(r'(\n)(\s+)', r'\1', p)
    p=re.sub(r'(\s+)(\n)', r'\2', p)
    p=re.sub(r'(\n)', r' ', p)
    
    z=0
    while( z<len(p) and (p[z].isalpha()!=True and p[z].isnumeric()!=True) ):
        z+=1
    
    p=p[z:]
    p=p.strip()
    p=re.sub(" +", " ", p)

    summary_sent.append(p)



leng=[]
print(summary_sent)
print(len((" ".join(sentences)).split(" ")))
leng.append(len((" ".join(sentences)).split(" ")))

finalS=[]
finalS.append(" ".join(summary_sent))

df_out_sentence=DataFrame(zip(finalS, leng), columns=['summary', 'leng'])
df_out_sentence.to_csv("summarySent_world_POS.csv", header=True, index=None)

16
['The 80-year-old three-time World Cup champion called it an unforgettable day and urged discipline to preserve lives.', 'The COVID-19 vaccine developed by Chinas Sinopharm has been approved for emergency use in the Maldives, the Maldives Food and Drug Administration announced at a press conference Monday afternoon.', 'Azerbaijan on Thursday received a batch of Sinovacs COVID-19 vaccines that it directly purchased from China.', 'Serbian President Aleksandar Vucic avucic received a dose of Chinese Sinopharm COVID-19 vaccine on Tuesday, encouraging more people to join the immunization, according to local media.', 'Delhi high court refuses to stay Covaxin trial among children (RichaBanka reports)', 'I request for your kind intervention so that an early approval is received for Covaxin from WHO', 'The U.S. drug regulator on Friday added a warning to the literature that accompanies Moderna and Pfizer-BioNTech COVID-19 vaccine shots, indicating the rare risk of heart inflammation after it